In [0]:
%sql
SELECT 
  get_json_object(response, '$.usage.prompt_tokens') AS prompt_tokens,
  get_json_object(response, '$.usage.completion_tokens') AS completion_tokens,
  get_json_object(response, '$.usage.total_tokens') AS total_tokens
FROM shm.marketing.marketing_agent_payload

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
import json

df = spark.table('shm.marketing.marketing_agent_payload')

@udf(IntegerType())
def extract_total_tokens(json_str):
    try:
        data = json.loads(json_str)
        # Recursively find all 'usage' dicts and sum 'total_tokens'
        def find_usages(obj):
            usages = []
            if isinstance(obj, dict):
                if 'usage' in obj and 'total_tokens' in obj['usage']:
                    usages.append(obj['usage']['total_tokens'])
                for v in obj.values():
                    usages.extend(find_usages(v))
            elif isinstance(obj, list):
                for item in obj:
                    usages.extend(find_usages(item))
            return usages
        return sum(find_usages(data))
    except Exception:
        return None

# Register and use the UDF
df = df.sample(1).withColumn("total_tokens", extract_total_tokens(df.response))
display(df.select('total_tokens'))

In [0]:
response_str = df.limit(1).collect()[0].asDict()['response']
data = json.loads(response_str)

In [0]:
def extract_total_tokens(data):
    try:
        # Recursively find all 'usage' dicts and sum 'total_tokens'
        def find_usages(obj):
            usages = []
            if isinstance(obj, dict):
                if 'usage' in obj and 'total_tokens' in obj['usage']:
                    usages.append(obj['usage']['total_tokens'])
                for v in obj.values():
                    usages.extend(find_usages(v))
            elif isinstance(obj, list):
                for item in obj:
                    usages.extend(find_usages(item))
            return usages
        return sum(find_usages(data))
    except Exception:
        return None

In [0]:
json.loads(data['databricks_output']['trace']['data']['spans'][-1]['attributes']['mlflow.spanOutputs'])['messages'][0]['response_metadata']['usage']